In [27]:
import csv
from scipy.constants import h,e,c
import numpy as np
import itertools

# Photoelectric Effect

In [28]:
def PhotoelectricEffect(stop_voltage:float, nu_0:float=None, work_function_eV:float=None):
    if nu_0 is not None and work_function_eV is not None:
        raise ValueError("You must give just one of either nu_0 or work_function_eV.")
    elif work_function_eV is not None:
        W = work_function_eV*1.60217733E-19
    else:
        W=h*nu_0
    return (e*stop_voltage+W)/h

In [29]:
frequency=PhotoelectricEffect(stop_voltage=0.01,work_function_eV=0.5)
print(f"{frequency/1e15:.2f}xe15 Hz")

0.12xe15 Hz


# Materials and voltages avaliable

In [30]:
work_function_materials={'Cu':4.53, 'Ca': 2.87,'Cs': 2.1 ,'Fe': 4.67, 'Co': 5.0, 'Lowest_Work_Function':1}
voltages = [2**(i/50) for i in range(-250,250)]

# Wavelength converter

In [31]:
def toWavelength(frequency:float):
    return c/frequency

In [32]:
wavelength=toWavelength(frequency)
print(f"*Frequency*: {frequency/1e15:.2f}xe15 Hz | *Wavelenght*: {wavelength*1e9:.2f} nm")

*Frequency*: 0.12xe15 Hz | *Wavelenght*: 2431.06 nm


# Defining the matrix

In [33]:
# fields=['Work Function','Voltage','Frequency','Wavelength',"Colour","Spectrum range"]
# matrix = [[i, j, PhotoelectricEffect(stop_voltage=j, work_function_eV=i),toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))] for i, j in itertools.product(work_function_materials.values(), voltages)]

# Writing the data into csv file

In [34]:
# with open('phe_data.csv', 'w') as csvfile: 
#     # creating a csv writer object 
#     csvwriter = csv.writer(csvfile) 
        
#     # writing the fields 
#     csvwriter.writerow(fields) 
#     # writing the data rows 
#     csvwriter.writerows(matrix)

# Checking the range

In [35]:
# npmatrix=np.array(matrix)
# minWl=toWavelength(min(npmatrix[:,2]))*1e9
# maxWl=toWavelength(max(npmatrix[:,2]))*1e9
# print(f"minWl =  {minWl:.2f} nm, maxWl = {maxWl:.2f} nm")


In [36]:
from typing import Tuple


def wave2rgb(wave:float)->Tuple[float,float,float,str]:
    # This is a port of javascript code from  http://stackoverflow.com/a/14917481
    gamma = 0.8
    intensity_max = 1

    if wave < 380:
        red, green, blue = -(wave - 440) / (440 - 380), 0, 1
    elif wave < 440:
        red = -(wave - 440) / (440 - 380)
        green, blue = 0, 1
    elif wave < 490:
        red = 0
        green = (wave - 440) / (490 - 440)
        blue = 1
    elif wave < 510:
        red, green = 0, 1
        blue = -(wave - 510) / (510 - 490)
    elif wave < 580:
        red = (wave - 510) / (580 - 510)
        green, blue = 1, 0
    elif wave < 645:
        red = 1
        green = -(wave - 645) / (645 - 580)
        blue = 0
    elif wave <= 780:
        red, green, blue = 1, 0, 0
    else:
        red, green, blue = 0, 0, 0

    # let the intensity fall of near the vision limits
    if wave < 380:
        factor = 0
        return 0.72,0.4,0.93,"UV"
    elif wave < 420:
        factor = 0.3 + 0.7 * (wave - 380) / (420 - 380)
    elif wave < 700:
        factor = 1
    elif wave <= 780:
        factor = 0.3 + 0.7 * (780 - wave) / (780 - 700)
    else:
        factor = 0
        return 1,0,0,"IR"

    def f(c):
        return 0 if c == 0 else intensity_max * pow (c * factor, gamma)

    return f(red), f(green), f(blue), "VIS"

In [37]:
import matplotlib
r,g,b,spec=wave2rgb(wavelength*1e9)
print(f"({r},{g},{b}), {spec})")
print(matplotlib.colors.to_hex([ r, g, b ]))

(1,0,0), IR)
#ff0000


In [50]:
fields=['Name','Material','Work Function','Voltage','Frequency','Wavelength',"Colour RGB","Colour Hex", "Spectrum range"]
val_list = list(work_function_materials.values())
key_list = list(work_function_materials.keys())
matrix1 = []
k=0
for i in work_function_materials.values():
    for j in voltages:
        r=wave2rgb(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9)[0]
        g=wave2rgb(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9)[1]
        b=wave2rgb(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9)[2]
        matrix1.append([k,key_list[val_list.index(i)],i, round(j,4), round(PhotoelectricEffect(stop_voltage=j, work_function_eV=i)/1e15,4),round(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9,4),f"(R={r},G={g},B={b},A=1)",matplotlib.colors.to_hex(wave2rgb(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9)[:3]),wave2rgb(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9)[3]])
        k=k+1
# matrix = [[key_list[val_list.index(i)],i, j, PhotoelectricEffect(stop_voltage=j, work_function_eV=i),toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i)),wave2rgb(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9)[:3],matplotlib.colors.to_hex(wave2rgb(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9)[:3]),wave2rgb(toWavelength(PhotoelectricEffect(stop_voltage=j, work_function_eV=i))*1e9)[3]] for i, j in itertools.product(work_function_materials.values(), voltages)]

In [51]:
with open('phe_data.csv', 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
    # writing the data rows 
    csvwriter.writerows(matrix1)